### Import all needed libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import chart_studio.plotly as py
import re

### Read in the csv file

In [2]:
nonstd = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/3.4-Incidents_Regex_AIRPORT_NONSTD_SET3.csv')
nonstd

,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,REMARKS,MORID.CEDAR,...,ACLOCATION.CEDAR,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,UASLOCATION,DATASET
0,2018-01-06,04:57,2018-01-06T04:57Z,N738UT,DEN,UAS,NaN,F11,Aircraft reported a large UAS with green light...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26 NM E of F11,AIRPORT
1,2018-01-15,07:35,2018-01-15T07:35Z,N6522,DEN,UAS,NaN,POC,Reported a UAS 400 feet below off the left sid...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 NM N of POC,AIRPORT
2,2018-01-22,01:25,2018-01-22T01:25Z,SWA4195,DEN,UAS,NaN,ELP,Aircraft reported a white UAS on the left side...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5 NM S of ELP,AIRPORT
3,2018-02-16,04:40,2018-02-16T04:40Z,N398ES,DEN,UAS,NaN,CCR,Aircraft reported a UAS 200 feet below and 500...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 NM N of CCR,AIRPORT
4,2018-02-19,00:39,2018-02-19T00:39Z,N26V,DEN,UAS,NaN,RSW,Aircraft reported a UAS pass behind their airc...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 NM N of RSW,AIRPORT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,2021-07-05,00:43,2021-07-05T00:43Z,N6719U,DEN,UAS,NaN,NCT,Aircraft observed a black UAS while ESE bound ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10 NM E of NCT,AIRPORT
405,2021-07-05,17:56,2021-07-05T17:56Z,RPA3715,DEN,UAS,NaN,SRQ,"Aircraft observed a UAS while on SRQ ramp, at ...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.5 NM N of SRQ,AIRPORT
406,2021-07-10,20:07,2021-07-10T20:07Z,KNIFE04,DEN,UAS,NaN,CNO,Aircraft observed a white quad-copter UAS at 1...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4 NM S of CNO,AIRPORT
407,2021-07-18,00:55,2021-07-18T00:55Z,NaN,DEN,UAS,NaN,YIP,"Aircraft observed a UAS while S bound at 2,000...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4 NM N of YIP,AIRPORT


### Exploring UAS locations

In [9]:
uas_loc = nonstd['UASLOCATION'] 

# If the regular expressions did not hit on any location information, pass it UNKN for now
for i in range(len(nonstd)):
    if len(uas_loc[i]) == 0:
        uas_loc[i] = 'UNKN'

In [50]:
uas_loc_nonNull = uas_loc[uas_loc != 'UNKN'].to_list()
uas_airport = []

for i in range(len(uas_loc_nonNull)):
    airport = uas_loc_nonNull[i].split(' ')[-1]
    
    if len(airport) <= 4:
        uas_airport.append(airport)
    else:
        trim_air = airport[-3:]
        uas_airport.append(trim_air)

In [51]:
uas_airport_df = pd.DataFrame()
uas_airport_df['IDENT'] = uas_airport
uas_airport_df

,IDENT
0,F11
1,POC
2,ELP
3,CCR
4,RSW
...,...
404,NCT
405,SRQ
406,CNO
407,YIP


### Read in the airports dataset

In [52]:
airportsC = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/airports_cleaned.csv')

### Join datasets together

In [57]:
uas_air_loc= pd.merge(uas_airport_df, airportsC, on='IDENT', how='left')
full_loc = pd.DataFrame()
full_loc['REMARKS'] = nonstd['REMARKS']
full_loc['UAS_LOC'] = nonstd['UASLOCATION']

final_df1 = pd.concat([full_loc, uas_air_loc], axis = 1)

In [58]:
final_df1

,REMARKS,UAS_LOC,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,Aircraft reported a large UAS with green light...,26 NM E of F11,F11,NaN,NaN,NaN,NaN
1,Reported a UAS 400 feet below off the left sid...,1.5 NM N of POC,POC,Brackett Fld,34.091667,-117.781778,KPOC
2,Aircraft reported a white UAS on the left side...,4.5 NM S of ELP,ELP,El Paso Intl,31.807333,-106.376361,KELP
3,Aircraft reported a UAS 200 feet below and 500...,5 NM N of CCR,CCR,Buchanan Fld,37.989657,-122.056902,KCCR
4,Aircraft reported a UAS pass behind their airc...,2 NM N of RSW,RSW,Southwest Florida Intl,26.536167,-81.755167,KRSW
...,...,...,...,...,...,...,...
404,Aircraft observed a black UAS while ESE bound ...,10 NM E of NCT,NCT,NaN,NaN,NaN,NaN
405,"Aircraft observed a UAS while on SRQ ramp, at ...",.5 NM N of SRQ,SRQ,Sarasota/Bradenton Intl,27.395444,-82.554389,KSRQ
406,Aircraft observed a white quad-copter UAS at 1...,4 NM S of CNO,CNO,Chino,33.974779,-117.636483,KCNO
407,"Aircraft observed a UAS while S bound at 2,000...",4 NM N of YIP,YIP,Willow Run,42.240028,-83.530889,KYIP


### Save the records that linked to airports into separate csv file

In [66]:
final = final_df1[final_df1['NAME'].notnull()]
final

,REMARKS,UAS_LOC,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID
1,Reported a UAS 400 feet below off the left sid...,1.5 NM N of POC,POC,Brackett Fld,34.091667,-117.781778,KPOC
2,Aircraft reported a white UAS on the left side...,4.5 NM S of ELP,ELP,El Paso Intl,31.807333,-106.376361,KELP
3,Aircraft reported a UAS 200 feet below and 500...,5 NM N of CCR,CCR,Buchanan Fld,37.989657,-122.056902,KCCR
4,Aircraft reported a UAS pass behind their airc...,2 NM N of RSW,RSW,Southwest Florida Intl,26.536167,-81.755167,KRSW
6,Aircraft reported a white quad copter UAS 200 ...,5 NM N of RSW,RSW,Southwest Florida Intl,26.536167,-81.755167,KRSW
...,...,...,...,...,...,...,...
401,"Aircraft observed a UAS while N bound at 1,200...",1.5 NM S of MTC,MTC,Selfridge ANGB,42.613894,-82.836910,KMTC
403,1448 EDT / 1848 UTC 06/29/2021\r\nN5088L obse...,1 NM W of ALB,ALB,Albany Intl,42.749116,-73.801980,KALB
405,"Aircraft observed a UAS while on SRQ ramp, at ...",.5 NM N of SRQ,SRQ,Sarasota/Bradenton Intl,27.395444,-82.554389,KSRQ
406,Aircraft observed a white quad-copter UAS at 1...,4 NM S of CNO,CNO,Chino,33.974779,-117.636483,KCNO


In [67]:
final.to_csv('3.4Nonstd_airpot_uas_loc.csv', index = False)

### Exploring the remaining 181 records

In [60]:
nonstd1 = final_df1[final_df1['NAME'].isnull()]
nonstd1

,REMARKS,UAS_LOC,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,Aircraft reported a large UAS with green light...,26 NM E of F11,F11,NaN,NaN,NaN,NaN
5,Aircraft reported a small blue and white UAS a...,25 NM N of ZJX,ZJX,NaN,NaN,NaN,NaN
8,Reported a UAS off the left side while S bound...,46 NM N of ZDC,ZDC,NaN,NaN,NaN,NaN
12,"Aircraft reported a white quad copter UAS 1,00...",15 NM S of I90,I90,NaN,NaN,NaN,NaN
13,"Aircraft reported a metallic UAS at 6,500 feet...",21 NM S of D10,D10,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
392,Aircraft observed a quadcopter UAS 100 feet be...,1 NM E of ZME,ZME,NaN,NaN,NaN,NaN
399,Marine Aircraft- Moonlight (ML) - ML31/MV22 - ...,11 NM N of SCT,SCT,NaN,NaN,NaN,NaN
402,Aircraft observed a silver and white quad-copt...,8 NM N of D01,D01,NaN,NaN,NaN,NaN
404,Aircraft observed a black UAS while ESE bound ...,10 NM E of NCT,NCT,NaN,NaN,NaN,NaN


### Read in the Navaid dataset

In [61]:
navaid = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/NAVAID_Cleaned.csv')
navaid

,LONGITUDE,LATITUDE,IDENT,NAME_TXT
0,-176.676024,51.871896,ADK,MOUNT MOFFETT
1,-151.744257,68.136626,AKP,ANAKTUVUK PASS
2,-149.960164,61.167868,TED,ANCHORAGE
3,-149.769204,61.255004,EDF,ELMENDORF
4,-159.597931,61.590169,ANI,ANIAK
...,...,...,...,...
1669,-156.788152,71.273426,BRW,BARROW
1670,-156.781302,71.282300,VIR,BROWERVILLE
1671,-75.164524,43.026523,UCA,UTICA
1672,-90.434470,40.937351,GBG,GALESBURG


### Join the remaining records with navaid dataset

In [62]:
join_uas_nav = pd.merge(nonstd1, navaid, on='IDENT', how='left')
join_uas_nav

,REMARKS,UAS_LOC,IDENT,NAME,LATITUDE_x,LONGITUDE_x,ICAO_ID,LONGITUDE_y,LATITUDE_y,NAME_TXT
0,Aircraft reported a large UAS with green light...,26 NM E of F11,F11,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aircraft reported a small blue and white UAS a...,25 NM N of ZJX,ZJX,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Reported a UAS off the left side while S bound...,46 NM N of ZDC,ZDC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Aircraft reported a white quad copter UAS 1,00...",15 NM S of I90,I90,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Aircraft reported a metallic UAS at 6,500 feet...",21 NM S of D10,D10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
176,Aircraft observed a quadcopter UAS 100 feet be...,1 NM E of ZME,ZME,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,Marine Aircraft- Moonlight (ML) - ML31/MV22 - ...,11 NM N of SCT,SCT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,Aircraft observed a silver and white quad-copt...,8 NM N of D01,D01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179,Aircraft observed a black UAS while ESE bound ...,10 NM E of NCT,NCT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Only one record was linked to navaid

In [63]:
nav_loc = join_uas_nav[join_uas_nav['NAME_TXT'].notnull()]
nav_loc

,REMARKS,UAS_LOC,IDENT,NAME,LATITUDE_x,LONGITUDE_x,ICAO_ID,LONGITUDE_y,LATITUDE_y,NAME_TXT
127,PIC reported a black UAS 200 feet below their ...,3 NM N of ZHU,ZHU,NaN,NaN,NaN,NaN,-73.346935,45.564084,HAUTS-BOIS


### Save the navaid record in a csv file

In [68]:
nav_loc.to_csv('3.4Nonstd_navaid_uas_loc.csv', index = False)

### We have 180 remaining records that are not airports nor navaids

In [64]:
len(join_uas_nav[join_uas_nav['NAME_TXT'].isnull()])

180

In [65]:
# Export remaining to csv file
remaining = join_uas_nav[join_uas_nav['NAME_TXT'].isnull()]
remaining.to_csv('3.4Non-std_uasLoc_noAir_noNav.csv', index = False)